In [1]:
from model.MF.MF import MatrixFactorization

from model.MF.preprocessing import ids_encoder


import pandas as pd
import movielens
import splitter
import mind

In [ ]:
df = movielens.load_pandas_df("1M")


/home/vinmike/Documents/GitHub/LLM4Rec-Dataloader/movielens.py:156: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(


In [ ]:
df

,userID,itemID,rating,timestamp
0,1,661,3.0,978302109
1,1,914,3.0,978301968
2,1,3408,4.0,978300275
3,1,2355,5.0,978824291
4,1,1197,3.0,978302268
...,...,...,...,...
1000203,6040,1091,1.0,956716541
1000204,6040,1094,5.0,956704887
1000205,6040,562,5.0,956704746
1000206,6040,1096,4.0,956715648


In [ ]:
ratings = pd.DataFrame(df[["userID", "itemID", "rating"]])

In [6]:
ratings.head()

,userID,itemID,rating
0,1,661,3.0
1,1,914,3.0
2,1,3408,4.0
3,1,2355,5.0
4,1,1197,3.0


In [7]:
# importing module
import sys

# printing all directories for 
# interpreter to search
sys.path

['/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/home/vinmike/.local/lib/python3.10/site-packages',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages']

In [ ]:

ratings, uencoder, iencoder = ids_encoder(ratings)

m = ratings["userID"].nunique()   # total number of users
n = ratings["itemID"].nunique()   # total number of items


In [ ]:
split_df = splitter.interactive_split(ratings)

[0.8, 0.2]


In [ ]:
split_df[1]

,userID,itemID,rating
0,0,3642,4.0
1,0,3234,5.0
2,0,1853,4.0
3,0,245,4.0
4,0,3432,3.0
...,...,...,...
200010,6039,1796,3.0
200011,6039,203,4.0
200012,6039,265,4.0
200013,6039,1016,5.0


In [ ]:
x_train = split_df[0][["userID", "itemID"]].to_numpy()
y_train = split_df[0][["rating"]].to_numpy().reshape(-1)

x_test = split_df[1][["userID", "itemID"]].to_numpy()
y_test = split_df[1][["rating"]].to_numpy().reshape(-1)

In [57]:
y_test

array([4., 5., 4., ..., 4., 5., 3.])

In [56]:
x_test

array([[   0, 3642],
       [   0, 3234],
       [   0, 1853],
       ...,
       [6039,  265],
       [6039, 1016],
       [6039,  952]])

In [58]:


MF = MatrixFactorization(m, n, k=10, alpha=0.01, lamb=1.5)

history = MF.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))



Training Matrix Factorization Model ...
k=10 	 alpha=0.01 	 lambda=1.5
epoch 1/10 - loss : 1.926 - val_loss : 1.934
epoch 2/10 - loss : 1.533 - val_loss : 1.537
epoch 3/10 - loss : 1.503 - val_loss : 1.506
epoch 4/10 - loss : 1.494 - val_loss : 1.497
epoch 5/10 - loss : 1.491 - val_loss : 1.493
epoch 6/10 - loss : 1.489 - val_loss : 1.491
epoch 7/10 - loss : 1.488 - val_loss : 1.49
epoch 8/10 - loss : 1.487 - val_loss : 1.49
epoch 9/10 - loss : 1.487 - val_loss : 1.489
epoch 10/10 - loss : 1.487 - val_loss : 1.489


In [59]:
MF.evaluate(x_test, y_test)

validation error : 1.489


1.4891639952848525

***MIND***

In [ ]:
behaviors_train_df, behaviors_dev_df, news_train_df, news_dev_df = mind.load_pandas_df('small')


In [33]:
from sklearn.preprocessing import LabelEncoder

def mind_ids_encoder(ratings, user_column="userID", item_column="history"):
    """
    Encodes the user and item IDs from the ratings dataframe.
    """
    users = sorted(ratings[user_column].unique())
    items = sorted(set([item for sublist in ratings[item_column].apply(lambda x: x.split()).tolist() for item in sublist]))

    # create users and items encoders
    uencoder = LabelEncoder()
    iencoder = LabelEncoder()

    # fit users and items ids to the corresponding encoder
    uencoder.fit(users)
    iencoder.fit(items)

    # encode userids and itemids
    ratings[user_column] = uencoder.transform(ratings[user_column].tolist())

    # Now we need to handle history, positive, and negative as items list
    ratings[item_column] = ratings[item_column].apply(lambda x: [iencoder.transform([i])[0] for i in x.split()])

    return ratings, uencoder, iencoder

def preprocess_mind_data(behaviors_df):
    """
    Preprocess MIND dataset to extract the relevant information
    and format it for Matrix Factorization.
    """
    ratings = pd.DataFrame(columns=['userID', 'itemID', 'rating'])

    # Replace NaN in 'history' column with empty string
    behaviors_df['history'] = behaviors_df['history'].fillna('')

    # Create implicit feedback from history and positive items
    new_rows = []  # List to store new rows to be concatenated
    for _, row in behaviors_df.iterrows():
        user_id = row['userID']
        history = row['history']  # This will now be an empty string if original was NaN
        positive = row['positive'].split()
        negative = row['negative'].split()

        # For implicit feedback, consider positive items as 1 (interacted), negative as 0 (not interacted)
        for item in history.split():  # Safely split, even if history is an empty string
            new_rows.append({'userID': user_id, 'itemID': item, 'rating': 1})
        
        for item in positive:
            new_rows.append({'userID': user_id, 'itemID': item, 'rating': 1})
        
        for item in negative:
            new_rows.append({'userID': user_id, 'itemID': item, 'rating': 0})

    # Convert new rows into DataFrame and concatenate with the existing ratings DataFrame
    new_rows_df = pd.DataFrame(new_rows)
    ratings = pd.concat([ratings, new_rows_df], ignore_index=True)

    # Encoding the user and item ids
    ratings, uencoder, iencoder = ids_encoder(ratings)

    return ratings, uencoder, iencoder


In [44]:
train_df, uencoder, iencoder = preprocess_mind_data(behaviors_train_df)


In [ ]:
dev_df, _, _ = preprocess_mind_data(behaviors_dev_df)


In [45]:
train_df

,userID,itemID,rating
0,2246,34394,1
1,2246,25006,1
2,2246,18802,1
3,2246,27282,1
4,2246,6417,1
...,...,...,...
10451833,32251,18139,1
10451834,32251,24378,0
10451835,32251,24378,0
10451836,32251,7345,0


In [ ]:

m = train_df["userID"].nunique()   # total number of users
n = train_df["itemID"].nunique()   # total number of items


50000

In [46]:
x_train = train_df[["userID", "itemID"]].to_numpy()
y_train = train_df[["rating"]].to_numpy().reshape(-1)

x_test = dev_df[["userID", "itemID"]].to_numpy()
y_test = dev_df[["rating"]].to_numpy().reshape(-1)


In [ ]:
# Instantiate and train the model
MF = MatrixFactorization(m, n, k=10, alpha=0.01, lamb=1.5)
history = MF.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))



In [ ]:
# Evaluate the model
MF.evaluate(x_test, y_test)